# Goals
1. predict for every store individually
- make stationary target by diff, yes/no?
- z score normalization on train data
- predict next 16 values directly vs recursively?
2. predict store individually but with every pair/family as parameter
- needs zscore normalization
- stationary target yes/no?
3. predict all store/family pairs simultaneously
- zscore? maybe not needed
- stationary?

features:
1. time features:
- linear timestamp
- sin/cos of year, check for (week/month) if pattern present
- encoding of weekday, maybe also month
2. oil/holidays/location should be ok


In [ ]:
import tensorflow as tf
tf.random.set_seed(42)
import numpy as np
np.random.seed(42)

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import plotly.offline as pyo
from plotly import subplots
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.tsa.api as smt

from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_log_error


from keras.models import Model
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense,Input,concatenate
from tensorflow import keras
from tensorflow.keras.layers import *


from baseFunctions import *
from data_helpers import processData6

In [ ]:
data, propDicts, flippedPropDicts = processData6()

In [ ]:
data1 = data.copy()

# feature engineering

aggregated data
- there is some linear trend 

In [ ]:
from scipy.signal import periodogram
def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette("husl", n_colors=X[period].nunique(),)
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax


def plot_periodogram(ts, cyclesPerUnit, n_domFreq = 15, detrend='linear', ax=None):
    freqencies, power = periodogram(
        ts,
        fs=cyclesPerUnit,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, power, color="purple")
    ax.set_xscale("log")
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")

    sorted_indices = np.argsort(power)[::-1]  # Sort indices in descending order of power
    dominant_freqs = freqencies[sorted_indices[:n_domFreq]]
    print(dominant_freqs)
    #return freqencies, power 

In [ ]:
dailyData = data1.groupby('date')['sales'].sum()

In [ ]:
dec = sm.tsa.seasonal_decompose(dailyData,period = 12, model = 'additive').plot()
plt.show()

In [ ]:
plot_periodogram(dailyData, 365, n_domFreq=30)

# strong frequencies     => TimePeriod 
# 52 (weekly) 365/52     = 7 days
# 24 (biweekly) 365/24   = 15 days (half-month)
# 104 (halfweek) 365/104 = 3.5 = 3.5 days 
# 12 (monthly)  365/12   = 30 days
# 6 (bimonthly)          = 60 days
# 4 (quarters)           = 90 days
# 3 (thirds)             = 120 days
# 2 (half-year)          = 182
# 1 (yearly)             = 365 

In [ ]:
tsplot(dailyData)

In [ ]:
data1['date'] = pd.to_datetime(data1['date'])
data1.date

In [ ]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess


# Create a DeterministicProcess object with the Fourier terms
dp = DeterministicProcess(index=data1.set_index('date').index, constant=True, 
                          fourier=CalendarFourier(freq="A", order=2))

# Get the deterministic terms
deterministic_terms = dp.in_sample()

In [ ]:
# add week feature
data1['weekday'] = data1['date'].dt.weekday
#pd.get_dummies(data1['weekday']) add one how encoding if necessary
data1['month'] = data1['date'].dt.month